# Row Reduction/Gaussian Elimination

**References:**
- Section 2.1.1 *Naive Gaussian elimination* of {cite}`Sauer`.
- Section 6.1 *Linear Systems of Equations* of {cite}`Burden-Faires`.
- Section 7.1 of {cite}`Chenney-Kincaid`.

## Introduction

The problem of solving a system of $n$ simultaneous linear equations in $n$ unknowns,
with matrix-vector form $A x = b$, is quite thoroughly understood as far as having a good general-purpose methods usable with any $n \times n$ matrix $A$: essentially, Gaussian elimination (or row-reduction) as seen in most linear algebra courses, combined with some modifications to stay well away from division by zero: *partial pivoting*.
Also, good robust software for this general case is readily available, for example in the Julia package `LinearAlgebra`.

Nevertheless, this basic algorithm can be very slow when $n$ is large – as it often is when dealing with differential equations (even more so with *partial* differential equations).
We will see that it requires about $n^3/3$ arithmetic operations.

Thus I will summarise the basic method of row reduction or Gaussian elimination, and then build on it with methods for doing things more robustly, and on methods for doing it faster in some important special cases:

1. When one has to solve many systems $A x^{(m)} = b^{(m)}$ with the same matrix $A$ but different right-hand side vectors $b^{(m)}.$

2. When $A$ is *banded*: most elements are zero, and all the non-zero elements $a_{i,j}$ are near the main diagonal: $|i - j|$ is far less than $n$. (*Aside on notation:* "far less than" is sometimes denoted $\ll$, as in $|i-j| \ll n$.)

3. When $A$ is *strictly diagonally dominant*: each diagonal element $a_{i,i}$ is larger in magnitude that the sum of the magnitudes of all other elements in the same row.

Other cases not (yet) discussed in this text are

4. When $A$ is *positive definite*: symmetric ($a_{i,j} = a_{j,i}$) and with all eigenvalues positive.
This last condition would seem hard to verify, since computing all the eigenvalues of $A$ is harder that solving $Ax = b$,
but there are important situations where this property is automatically guaranteed, such as with *Galerkin* and *finite-element methods* for solving boundary value problems for differential equations.

5. When $A$ is *sparse*: most elements are zero, but not necessarily with all the non-zero elements near the main diagonal.

## Strategy for getting from mathematical facts to a good algorithm and then to its implentation in [Julia] code

Here I take the opportunity to illustrate some useful strategies for getting from mathematical facts and ideas to good algorithms and working code for solving a numerical problem.
The pattern we will see here, and often later, is:

### Step 1. Get a basic algorithm:
1. Start with mathematical facts (like the equations $\sum_{j=1}^n a_{ij}x_j = b_i$).
2. Solve to get an equation for each unknown — or for an updated aproximation of each unknown — in terms of other quantitities.
3. Specify an order of evaluation in which all the quantities at right are evaluated earlier.

In this, it is often best to start with a verbal description before specifying the details in more precise and detailed mathematical form.

### Step 2. Refine to get a more **robust** algorithm:
1. Identify cases that can lead to failure due to division by zero and such, and revise to avoid them.
2. Avoid inaccuracy due to problems like severe rounding error. One rule of thumb is that anywhere that a zero value is a fatal flaw (in particular, division by zero), a very small value is also a hazard when rounding error is present.
So **avoid very small denominators**. (We will soon examine this through the phenomenon of **loss of significance** and it extreme case **catastrophic cancellation**.)

### Step 3. Refine to get a more **efficient** algorithm
For example,
- Avoid repeated evaluation of exactly the same quantity.
- Avoid redundant calculations, such as ones whose value can be determnied in advance;
for example, values that can be shown in advance to be zero.
- Compare and choose between alternative algorithms.

## Gaussian elimination, a.k.a. row reduction

We start by considering the most basic algorithm, based on ideas seen in a linear algebra course.

The problem is best stated as a collection of equations for individual numerical values:

Given coefficients $a_{i,j} 1 \leq i \leq n,\, 1 \leq j \leq n$ and right-hand side values $b_i,\, 1 \leq i \leq n$,
solve for the $n$ unknowns $x_j,\, 1 \leq j \leq n$ in the equations
$$
\sum_{j=1}^n a_{i,j} x_j = b_i,\, 1 \leq i \leq n.
$$

In verbal form, the basic strategy of *row reduction* or *Gaussian elimination* is this:

- **Choose** one equation and use it to eliminate one **chosen** unknown from all the other equations, leaving that chosen equation plus $n-1$ equations in $n-1$ unknowns.
- Repeat recursively, at each stage using one of the remaining equations to eliminate one of the remaining unknowns from all the other equations.
- This gives a final equation in just one unknown, preceeded by an equation in that unknown plus one other, and so on: solve them in this order, from last to first.

### Determining those choices, to produce a first algorithm: "naive gaussian elimination"

A precise algorithm must include rules specifying all the choices indicated above.
The simplest "naive" choice, which works in most but not all cases, is to eliminate from the top to bottom and left to right:

- Use the first equation to eliminate the first unknown from all other equations.
- Repeat recursively, at each stage using the first remaining equation to eliminate the first remaining unknown. Thus, at step $k$, equation $k$ is used to eliminate unknown $x_k$.
- This gives one equation in just the last unknown $x_n$; another equation in the last two unknowns $x_{n-1}$ and $x_n$, and so on: solve them in this reverse order, evaluating the unknowns from last to first.

This usually works, but can fail because at some stage the (updated) $k$-th equation might not include the $k$-th unknown: that is, its coefficient might be zero, leading to division by zero.

We will refine the algorithm to deal with that in the later section {doc}`linear-equations-2-pivoting`.

## The general case of solving $Ax = b$

The problem of solving $Ax = b$ in general, when all you know is that $A$ is an $n \times n$ matrix and $b$ is an $n$-vector, can in most cases be handled well by using standard software rather than by writing your own code. Here is an example in Julia, solving

$$
\left[ \begin{array}{rrr} 4 & 2 & 7 \\ 3 & 5 & -6 \\ 1 & -3 & 2 \end{array} \right]
\left[ \begin{array}{r} x_1 \\ x_2 \\ x_3 \end{array} \right]
= \left[ \begin{array}{r} 2 \\ 3 \\ 4 \end{array} \right]
$$

In [1]:
A = [4.0 2.0 7.0; 3.0 5.0 -6.0; 1.0 -3.0 2.0]
println("A =\n$(A)")
b = [2.0; 3.0; 4.0]
println("b = $(b)")
println("A*b = $(A*b)")

A =
[4.0 2.0 7.0; 3.0 5.0 -6.0; 1.0 -3.0 2.0]
b = [2.0, 3.0, 4.0]
A*b = [42.0, -3.0, 1.0]


```{prf:remark} On Julia
:label: remark-julia-arrays

- See the notes on {ref}`julia-language-arrays` or [](julia-language-arrays) in {doc}`julia-language-notes`.

- Julia mimics Matlab's notation for "dividing from the left":
the solution of $Ax = b$ is $x = A^{-1} b$ and given by `A\b`; it is not $b A^{-1}$ which is what you get from the usual "divide from the right" notation of `b/A`.
```

See the notes on {ref}`julia-language-arrays` or [Julia arrays](julia-language-arrays) or [](julia-language-arrays)  in {doc}`julia-language-notes`.

In [2]:
x = A\b;

In [3]:
println("Julia says that the solution of Ax=b is x=$(x)")

Julia says that the solution of Ax=b is x=[1.8116883116883116, -1.0324675324675323, -0.45454545454545453]


Check the **residual** $b - Ax$, a measure of *backward error*:

In [4]:
r = b-A*x;

In [5]:
println()
println("As a check, the residual is")
println("    r = b - Ax = $(r)")
println("and its infinity (or 'maximum') norm is")
println("    ||r|| = $(maximum(abs.(r)))")


As a check, the residual is
    r = b - Ax = [0.0, 0.0, 8.881784197001252e-16]
and its infinity (or 'maximum') norm is
    ||r|| = 8.881784197001252e-16


```{prf:remark} Not quite zero values and rounding
:label: remark-1-not-quite-zero-values-and-rounding

Some values here that you might hope to be zero are instead very small non-zero numbers, with exponent $10^{-16}$,
due to rounding error in computer arithmetic.
For details on this (like why "-16" in particular) see {doc}`machine-numbers-rounding-error-and-error-propagation`.
```

## The naive Gaussian elimination algorithm, in pseudo-code

Here the elements of the transformed matrix and vector after step $k$ are named $a_{i,j}^{(k)}$ and $b_{k}^{(k)}$, so that the original values are $a_{i,j}^{(0)} = a_{i,j}$ and $b_{i}^{(0)} = b_{i}$.

The name $l_{i,k}$ is given to the multiple of row $k$ that is subtracted from row $i$ at step $k$. This naming might seem redundant, but it becomes very useful later.

```{prf:algorithm} naive Gaussian elimination
:label: naive-gaussian-elimination

for k from 1 to n-1 $\qquad$ *Step k: get zeros in column k below row k:*
<br>
$\quad$ for i from k+1 to n
<br>
$\qquad$ *Evaluate the multiple of row k to subtract from row i:*
<br>
$\quad\quad l_{i,k} = a_{i,k}^{(k-1)}/a_{k,k}^{(k-1)}$ $\qquad$ **If** $a_{k,k}^{(k-1)} \neq 0$!
<br>
$\qquad$ *Subtract $(l_{i,k}$ times row k) from row i in matrix A ...:*
<br>
$\quad\quad$ for j from 1 to n
<br>
$\quad\quad\quad a_{i,j}^{(k)} = a_{i,j}^{(k-1)} - l_{i,k} a_{k,j}^{(k-1)}$
<br>
$\quad\quad$ end
<br>
$\qquad$ ... and at right, subtract $(l_{i,k}$ times $b_k)$ from $b_i$:
<br>
$\quad\quad b_i^{(k)} = b_i^{(k-1)} - l_{i,k} b_{k}^{(k-1)}$ 
<br>
$\quad$ end
```

The rows before $i=k$ are unchanged, so they are ommited from the update;
however, in a situation where we need to complete the definitions of $A^{(k)}$ and $b^{(k)}$ we would also need the following inside the `for k` loop:

```{prf:algorithm} Inserting the zeros below the main diagonal
:label: gaussian-elimination-inserting-zeros

$\quad$ for i from 1 to k
<br>
$\quad\quad$ for j from 1 to n
<br>
$\quad\quad\quad a_{i,j}^{(k)} = a_{i,j}^{(k-1)}$
<br>
$\quad\quad$ end
<br>
$\quad\quad b_i^{(k)} = b_i^{(k-1)}$
<br>
$\quad$ end
```

However, the algorithm will usually be implemented by overwriting the previous values in an array with new ones, and then this part is redundant.

The next improvement in efficiency: the updates in the first $k$ columns at step $k$ give zero values (that is the key idea of the algorithm!),
so there is no need to compute or store those zeros,
and thus the only calculations needed in the above `for j from 1 to n` loop are covered by `for j from k+1 to n`.
Thus from now on we use only the latter—except when, for demonstration purposes, we need those zeros.

Thus, the standard algorithm looks like this:

```{prf:algorithm} basic Gaussian elimination
:label: gaussian-elimination

for k from 1 to n-1 $\qquad$ *Step k: Get zeros in column k below row k:*
<br>
$\quad$ for i from k+1 to n $\qquad$ *Update only the rows that change: from k+1 on:*
<br>
$\qquad$ *Evaluate the multiple of row k to subtract from row i:*
<br>
$\quad\quad l_{i,k} = a_{i,k}^{(k-1)}/a_{k,k}^{(k-1)}$ $\qquad$ **If** $a_{k,k}^{(k-1)} \neq 0$!
<br>
$\qquad$ *Subtract $(l_{i,k}$ times row k) from row i in matrix A, in the columns that are not automaticaly zero:*
<br>
$\quad\quad$ for j from k+1 to n
<br>
$\quad\quad\quad a_{i,j}^{(k)} = a_{i,j}^{(k-1)} - l_{i,k} a_{k,j}^{(k-1)}$
<br>
$\quad\quad$ end
<br>
$\qquad$ *and at right, subtract $(l_{i,k}$ times $b_k)$ from $b_i$:*
<br>
$\quad\quad b_i^{(k)} = b_i^{(k-1)} - l_{i,k} b_{k}^{(k-1)}$ 
<br>
$\quad$ end
```

## The naive Gaussian elimination algorithm, in Julia

Conversion to actual Julia code is now quite straightforward; there is litle more to be done than:

- Change the way that indices are described, from $b_i$ to `b[i]` and from $a_{i,j}$ to `A[i,j]`.

- Use case consistently in array names, since the quirk in mathematical notation of using upper-case letters for matrix names but lower case letters for their elements is gone!
In these notes, matrix names will be upper-case and vector names will be lower-case (even when a vector is considered as 1-column matrix).

- Rather than create a new array for each matrix $A^{(0)}$, $A^{(1)}$, etc. and each vector $b^{(0)}$, $b^{(1)}$,
we overwite each in the same array.

    for k in 1:n
        for i in k+1:n
            L[i,k] = A[i,k] / A[k,k]
            for j in k+1:n
                A[i,j] -= L[i,k] * A[k,j]
            end
            b[i] -= L[i,k] * b[k]
        end
    end

To demonstrate this, some additions are needed:
- Putting this algorithm into a function.
- Getting the value $n$ needed for the loop, using the fact that it is the length of vector `b`.
- Creating the array $L$.
- Copying the input arrays `A` and `b` into new ones, `U` and `c`, so that the original arrays are not changed. That is, when the row reduction is completed, `U`  contains $A^{(n-1)}$ and `c` contains $b^{(n-1)}$.

Also, for some demonstrations, the zero values below the main diagonal of `U` are inserted, though usually they would not be needed.

In [6]:
function rowreduce(A, b)
    # To avoid modifying the matrix and vector specified as input,
    # they are copied to new arrays, with the function copy().
    # Warning: it does not work to say "U = A" and "c = b";
    # this makes these names synonyms, referring to the same stored data.

    U = copy(A)  # not "U=A", which makes U and A synonyms
    c = copy(b)
    n = length(b)
    L = zeros(n, n)
    for k in 1:n-1
        for i in k+1:n
            # compute all the L values for column k:
            L[i,k] = U[i,k] / U[k,k]  # Beware the case where U[k,k] is 0
            for j in k+1:n
                U[i,j] -= L[i,k] * U[k,j]
            end
            # Put in the zeros below the main diagonal in column k of U;
            # this is not important for calculations,
            # since those elements of U are not used in backward substitution,
            # but it helps for displaying results and for checking the results via residuals.
            U[i,k] = 0.
            
            c[i] -= L[i,k] * c[k]
        end
    end
    for i in 2:n
        for j in 1:i-1
            U[i,j] = 0.
        end
    end
    return (U, c)
end;

Here is a helper function for displaying matrices.
Since it will be used in several sections, it is also in the module `NumericalMethods`, along with the above function;
see {doc}`NumericalMethods`.

```{prf:remark} Julia Modules
:label: julia-modules

Modules and their usage are introduced in the notes on [Using modules and packages](julia-language-notes#modules).
As explained there, these two functions can be obtained with

    include("NumericalMethods.jl")
    using .NumericalMethods: rowreduce
    using .NumericalMethods: printmatrix

Details about creating your own modules will be given in [creating modules](julia-language-notes#creatingmodules) once those notes are written;
meanwhile, examining {doc}`NumericalMethods` could help:
the actual module definition file is just the Julia code from there with the interspersed explanatory comments removed.
```

 Some helper functions to cleanup output

In [7]:
function printmatrix(A)
    # A helper function to "pretty print" matrices
    
    (rows, cols) = size(A)
    print("[ ")
    for row in 1:rows
        if row > 1
            print("  ")
        end
        for col in 1:cols
            print(A[row,col], " ")
        end
        if row < rows;
            println()
        else
            println("]")
        end
    end
end;

In [8]:
# A shorthand for rounding off to n significant digits.
import Base: round
round(x, n::Integer) = round(x, sigdigits=n);

In [9]:
println("A is")
printmatrix(A)
println("b = $(b)")

A is
[ 4.0 2.0 7.0 
  3.0 5.0 -6.0 
  1.0 -3.0 2.0 ]
b = [2.0, 3.0, 4.0]


In [10]:
(U, c) = rowreduce(A, b);

In [11]:
println("Row reduction gives")
println("U =")
printmatrix(U)
println("c = $(c)")

Row reduction gives
U =
[ 4.0 2.0 7.0 
  0.0 3.5 -11.25 
  0.0 0.0 -11.0 ]
c = [2.0, 1.5, 5.0]


Let's take advantage of the fact that we have used Julia's built-in linear algebra command `b\A` to get a very accurate approximation of the solution $x$ to $Ax=b$; this should also solve $Ux=c$, so check the backward error, a.k.a. the *residual*:

In [12]:
r = c - U * x
println("The residual (backward error) r = c-Ux is $(round.(r,4))," *
    " with maximum norm $(round(maximum(abs.(r)),4))")

The residual (backward error) r = c-Ux is [0.0, -2.22e-16, 0.0], with maximum norm 2.22e-16


```{prf:remark} Array slicing in Julia
:label: julia-array-slicing

Many operations in linear algebra can be expressed more concisely using
[array slicing](julia-language-notes#arrayindexingslicing) and
[vectorization](julia-language-notes#vectorization) as described in {doc}`julia-language-notes`
allowing the loops above to be expressed as

    for k in 1:n
        L[k+1:end,end] = A[k+1:end,k] / A[k,k]
        A[k+1:end,k+1:end] -= L[k+1:end,k] * A[[k],k+1:end]
        b[k+1:end] -= L[k+1:end,k] * b[k]
        end
    end
```

```{prf:remark} Matrix slicing in Julia
:label: julia-array-slicing-2

One subtlety here, as mentioned in the notes on [slicing](julia-language-notes#arrayindexingslicing):
that row slice at the end of line 3 has to be done as `A[[k],k+1:end]` with brackets around the row index,
in order to make it a 1-row matrix;
using `A[k,k+1:end]` instead would give a vector, and then the product would fail.
```

I will break my usual guideline of non-repetition by redefining `rowreduce`,
since this is just a restatement of exactly the same algorithm with different Julia notation.

While I am about it, I add a `demomode`, for display of intermediate results.

In [13]:
function rowreduce(A, b; demomode=false)
    # To avoid modifying the matrix and vector specified as input,
    # they are copied to new arrays, with the method .copy()
    # Warning: it does not work to say "U = A" and "c = b";
    # this makes these names synonyms, referring to the same stored data.
    
    # This version vectorizes the inner loops, and all of the "i, j" loop for updating U.

    if demomode
        println("rowreduce version 2: some loops vectorized")
    end
    U = copy(A)  # not "U=A", which makes U and A synonyms
    c = copy(b)
    n = length(b)
    L = zeros(n, n)
    for k in 1:n-1
        if demomode; println("Step $(k):"); end
        # compute all the L values for column k:
        L[k+1:end,k] = U[k+1:end,k] / U[k,k]  # Beware the case where U[k,k] is 0
        U[k+1:end,k+1:end] -= L[k+1:end,k] * U[[k],k+1:end]
        c[k+1:end] -= L[k+1:end,k] * c[k]
        
        # Insert the below-diagonal zeros in column k;
        # this is not important for calculations,
        # since those elements of U are not used in backward substitution,
        # but it helps for displaying results and for checking the results via residuals.
        U[k+1:end,k] .= 0.0

        if demomode
            println("After step $k the matrix is")
            printmatrix(U)
            println("and the right-hand side is $c")
        end
    end
     return (U, c)
end;

Repeating the above testing:

In [14]:
U = ones(1,3)

1×3 Matrix{Float64}:
 1.0  1.0  1.0

In [15]:
(U, c) = rowreduce(A, b, demomode=true);
println("Row reduction gives U=")
printmatrix(U)
println("and right-hand side $c")

rowreduce version 2: some loops vectorized
Step 1:
After step 1 the matrix is
[ 4.0 2.0 7.0 
  0.0 3.5 -11.25 
  0.0 -3.5 0.25 ]
and the right-hand side is [2.0, 1.5, 3.5]
Step 2:
After step 2 the matrix is
[ 4.0 2.0 7.0 
  0.0 3.5 -11.25 
  0.0 0.0 -11.0 ]
and the right-hand side is [2.0, 1.5, 5.0]
Row reduction gives U=
[ 4.0 2.0 7.0 
  0.0 3.5 -11.25 
  0.0 0.0 -11.0 ]
and right-hand side [2.0, 1.5, 5.0]


## Backward substitution with an upper triangular matrix

The transformed equations have the form

$$
\begin{split}
u_{1,1} x_1 + u_{1,2} x_2 +  u_{1,3} x_3 + \cdots  + u_{1,n} x_n &= c_1 \\
\vdots \\
u_{i,i} x_i + u_{i+1,i+1} x_{i+1} + \cdots + u_{i,n} x_n &= c_i \\
\vdots \\
u_{n-1,n-1} x_{n-1} + u_{n-1,n} x_{n} &= c_{n-1} \\
u_{nn} x_n &= c_n \\
\end{split}
$$

and can be solved from bottom up, starting with $x_n = c_n/u_{n,n}$.

All but the last equation can be written as

$$
u_{i,i}x_i + \sum_{j=i+1}^{n} u_{i,j} x_j = c_i, \; 1 \leq i \leq n-1
$$

and so solved as

$$
x_i = \frac{c_i - \sum_{j=i+1}^{n} u_{i,j} x_j}{u_{i,i}},
\qquad \textbf{ If } u_{i,i} \neq 0
$$

This procedure is **backward substitution**, giving the algorithm
```{prf:algorithm} Backward substitution
:label: backward-substitution

$x_n = c_n/u_{n,n}$
<br>for i from n-1 down to 1
<br>$\displaystyle \quad x_i = \frac{c_i - \sum_{j=i+1}^{n} u_{i,j} x_j}{u_{i,i}}$
<br>end
```

This works so long as none of the main diagonal terms $u_{i,i}$ is zero, because when done in this order, everything on the right hand side is known by the time it is evaluated.

For future reference, note that the elements $u_{k,k}$ that must be non-zero here, the ones on the **main diagonal** of $U$, are the same as the elements $a_{k,k}^{(k)}$ that must be non-zero in the row reduction stage above, because after stage $k$, the elements of row $k$ do not change any more: $a_{k,k}^{(k)} = a_{k,k}^{(n-1)} = u_{k,k}$.

```{prf:remark} Summing in Julia
:label: julia-summing

For an $n$-element single-index array `v`, the sum of its elements $\sum_{i=1}^{n} v_i$ is given by `sum(v)`.

Thus $\sum_{i=a}^{b} v_i$, the sum over a subset of indices $[a,b]$, is given by `sum(v[a:b])`.
```

### The backward substitution algorithm in Julia

With all the above Julia details, the core code for backward substitution is:

    x[end] = c[end]/U[end,end]
    for i in n-1:-1:1
        x[i] = (c[i] - U[i,i+1:end] * x[i+1:end])) / U[i,i]
    end

```{prf:observation} 
:label: remark-2-lu-is-functionally-correct

Note that the backward substitution algorithm and its Julia coding have a nice mathematical advantage over the row reduction algorithm above:
the precise mathematical statement of the algorithm does not need any intermediate quantities distinguished by superscripts ${}^{(k)}$
and correspondingly, all variables in the code have fixed meanings, rather than changing at each step.

In other words, all uses of the equal sign are mathematically correct as equations!

This can be advantageous in creating algorithms and code that is more understandable and more readily verified to be correct,
and is an aspect of the *functional programming* approach.
We will soon go part way to that *functional* ideal, by rephrasing Gaussian elimination in a form where all variables have clear,
fixed meanings, corresponding to the natural mathematical description of the process:
the method of **LU factorization** introduced in {doc}`linear-equations-3-lu-factorization`.
```

As a final demonstration, we put this second version of the code into a complete working Julia function and test it:

In [16]:
function backwardsubstitution(U, c; demomode=false)
    n = length(c)
    x = zeros(n)
    x[end] = c[end]/U[end,end]
    if demomode
        println("x_$n = $(x[n])")
    end
    for i in n-1:-1:1
        if demomode
            println("i=$i")
        end
        x[i] = ( c[i] - sum(U[i,i+1:end] .* x[i+1:end]) ) / U[i,i]
        if demomode
            println("x_$i = $(x[i])")
        end
    end
    return x
end;

```{prf:remark} 
:label: remark-importing-backwardsubstitution

As usual, this is also available via

    using .NumericalMethods: backwardsubstitution
```

In [17]:
x = backwardsubstitution(U, c, demomode=true)
println("x = $x")

x_3 = -0.45454545454545453
i=2
x_2 = -1.0324675324675323
i=1
x_1 = 1.8116883116883116
x = [1.8116883116883116, -1.0324675324675323, -0.45454545454545453]


In [18]:
println("x = $x")
r = b - A*x
println("The residual b - Ax = $(round.(r,4)), with maximum norm $(round(maximum(abs.(r)),4)))")

x = [1.8116883116883116, -1.0324675324675323, -0.45454545454545453]
The residual b - Ax = [0.0, 0.0, 8.882e-16], with maximum norm 8.882e-16)


Since one is often just interested in the solution given by the two steps of row reduction and then backward substitution,
they can be combined in a single function by composition:

In [19]:
solvelinearsystem(A, b) = backwardsubstitution(rowreduce(A, b)...);

```{prf:remark} The Julia "splat" operation
:label: julia-splat

The **splat** notation "..." takes the item to its left (here the tuple `(U, c)` returned by `rowreduce`) and unpacks it into separate items [here `U` and `c`, as needed for input to `backwardsubstitution`.
```

## Two code testing hacks: starting from a known solution, and using randomly generated examples

An often useful strategy in developing and testing code is to create a test case with a known solution;
another is to use random numbers to avoid accidently using a test case that in unusually easy.

```{prf:remark} function rand! from module Random
:label: julia-Random-rand

- The preferred style is to have all `import` and `using` statements near the top,
but since this is the first time we've heard of module `Random` I did not want it to be mentioned mysteriously above.

- The exclamation point in `rand!` indicates that the function modifies its input.
This will be discussed in [Functions part 2](julia-language-notes#functions2) when that section gets written.
```

In [20]:
using Random: rand!

In [21]:
n = length(b)
xrandom = zeros(n)
rand!(xrandom)  # fill with random values, uniform in [0, 1)
(xrandom *= 2.0) .-= 1.0  # double and then subtract 1: now uniform in [-1, 1)
println("xrandom = $xrandom")

xrandom = [0.13386638477695145, -0.7305805960100351, -0.8909949807354525]


Create a right-hand side b that automatically makes `xrandom` the correct solution:

In [22]:
brandom = A * xrandom;

In [23]:
println("A is"); printmatrix(A)
println("\nbrandom is $brandom")
(U, crandom) = rowreduce(A, brandom)
println("\nU is"); printmatrix(U)
println("\nResidual crandom - U*xrandom  = $(round.(crandom - U*xrandom,4))")
xcomputed = backwardsubstitution(U, crandom)
println("\nxcomputed is $(round.(xcomputed,12))")
r = brandom -  A*xcomputed
println("\nResidual brandom-A*xcomputed is $(round.(r,4))")
println("\nBackward error is $(round(maximum(abs.(r)),4))")
xerror = xrandom - xcomputed
println("\nError xrandom - xcomputed is $(round.(xerror,4))")
println("\nAbsolute error |xrandom-xcomputed| is $(round(maximum(abs.(xerror)),4))")

A is
[ 4.0 2.0 7.0 
  3.0 5.0 -6.0 
  1.0 -3.0 2.0 ]

brandom is [-7.162660518060432, 2.0946660586933934, 0.5436182113361516]

U is
[ 4.0 2.0 7.0 
  0.0 3.5 -11.25 
  0.0 0.0 -11.0 ]

Residual crandom - U*xrandom  = [0.0, 0.0, 0.0]

xcomputed is [0.133866384777, -0.73058059601, -0.890994980735]

Residual brandom-A*xcomputed is [0.0, 4.441e-16, -4.441e-16]

Backward error is 4.441e-16

Error xrandom - xcomputed is [0.0, 1.11e-16, 0.0]

Absolute error |xrandom-xcomputed| is 1.11e-16


## What can go wrong? Some examples

```{prf:example} An obvious division by zero problem
:label: example-obvious-division-by-zero

Consider the system of two equations

\begin{align*}
x_2 &= 1
\\
x_1 + x_2 &= 2
\end{align*}

It is easy to see that this has the solution $x_1 = x_2 = 1$;
in fact it is already in "reduced form".
However when put into matrix form

$$
\left[\begin{array}{rr} 0 & 1 \\ 1 & 1 \end{array}\right]
\left[\begin{array}{r} x_1 \\ x_2 \end{array}\right] = \left[\begin{array}{r} 1 \\ 2 \end{array}\right]
$$

the above algorithm fails, because the fist *pivot element* $a_{11}$ is zero:
```

In [24]:
A1 = [0.0 1.0 ; 1.0 1.0]
b1 = [1.0 ; 1.0]

println("A1 is)");   printmatrix(A1)
println("b1 is $(b1)")

A1 is)
[ 0.0 1.0 
  1.0 1.0 ]
b1 is [1.0, 1.0]


In [25]:
(U1, c1) = rowreduce(A1, b1)
x1 = backwardsubstitution(U1, c1)

println("U1 is"); printmatrix(U1)
print("c1 is $c1")
print("x1 is $x1")

U1 is
[ 0.0 1.0 
  0.0 -Inf ]
c1 is [1.0, -Inf]x1 is [NaN, NaN]

```{prf:remark} IEEE fake numbers Inf and NaN
:label: inf-nan

- `Inf`, meaning "infinity", is a special value given as the result of calculations like division by zero.
Surprisingly, it can have a sign!
- `NaN`, meaning "Not a Number", is a special value given as the result of a calculation like `0/0`.
```

```{prf:example} A less obvious division by zero problem
:label: example-less-obvious-division-by-zero

Next consider this system

$$
\left[\begin{array}{rrr} 1 & 1 & 1 \\ 1 & 1 & 2 \\ 1 & 2 & 2 \end{array}\right]
\left[\begin{array}{r} x_1 \\ x_2 \\ x_3 \end{array}\right] = \left[\begin{array}{r} 3 \\ 4 \\ 5 \end{array}\right]
$$

The solution is $x_1 = x_2 = x_3 = 1$, and this time none of th diagonal elements is zero,
so it is not so obvious that a division by zero problem will occur, but:
```

In [26]:
A2 = [1.0  1.0  1.0 ; 1.0  1.0  2.0 ; 1.0 2.0 2.]
b2 = [3.0 ; 4.0 ; 5.]

println("A2 is"); printmatrix(A2)
println("b2 is $b2")

A2 is
[ 1.0 1.0 1.0 
  1.0 1.0 2.0 
  1.0 2.0 2.0 ]
b2 is [3.0, 4.0, 5.0]


In [27]:
(U2, c2) = rowreduce(A2, b2)
x2 = backwardsubstitution(U2, c2)

println("U2 is"); printmatrix(U2)
println("c2 is $c2")
println("x2 is $x2")

U2 is
[ 1.0 1.0 1.0 
  0.0 0.0 1.0 
  0.0 0.0 -Inf ]
c2 is [3.0, 1.0, -Inf]
x2 is [NaN, NaN, NaN]


What happens here is that the first stage subtracts the first row from each of the others ...

In [28]:
A2[2,:] -= A2[1,:]
b2[2] -= b2[1]
A2[3,:] -= A2[1,:]
b2[3] -= b2[1];

... and the new matrix has the same problem as above at the next stage:

In [29]:
println("Now A2 is"); printmatrix(A2)
println("and b2 is $(b2)")

Now A2 is
[ 1.0 1.0 1.0 
  0.0 0.0 1.0 
  0.0 1.0 1.0 ]
and b2 is [3.0, 1.0, 2.0]


Thus, the second and third equations are

$$
\left[\begin{array}{rr} 0 & 1 \\ 1 & 1 \end{array}\right]
\left[\begin{array}{r} x_2 \\ x_3 \end{array}\right] = \left[\begin{array}{r} 1 \\ 2 \end{array}\right]
$$

with the same problem as in {prf:ref}`example-obvious-division-by-zero`.

```{prf:example} Problems caused by inexact arithmetic: "divison by almost zero"
:label: example-almost-division-by-zero

The equations

$$
\left[\begin{array}{rr} 1 & 10^{16} \\ 1 & 1 \end{array}\right]
\left[\begin{array}{r} x_1 \\ x_2 \end{array}\right] = \left[\begin{array}{r} 1+10^{16} \\ 2 \end{array}\right]
$$

again have the solution $x_1 = x_2 = 1$, and the only division that happens in the above algorithm for row reduction is by that pivot element
$a_{11} = 1, \neq 0$, so with exact arithmetic, all would be well. But:
```

In [30]:
A3 = [1.0  1e16 ; 1.0 1.0]
b3 = [1.0 + 1e16 ; 2.0]

println("A3 is"); printmatrix(A3)
println("b3 is $b3")

A3 is
[ 1.0 1.0e16 
  1.0 1.0 ]
b3 is [1.0e16, 2.0]


In [31]:
(U3, c3) = rowreduce(A3, b3)
x3 = backwardsubstitution(U3, c3)

println("U3 is"); printmatrix(U3)
println("c3 is $c3")
println("x3 is $x3")

U3 is
[ 1.0 1.0e16 
  0.0 -1.0e16 ]
c3 is [1.0e16, -9.999999999999998e15]
x3 is [2.0, 0.9999999999999998]


This gets $x_2 = 1$ fairly accurately, but $x_1$ is completely wrong!

One hint is that $b_1$, which should be $1 + 10^{16} = 1000000000000001$, is instead just given as $10^{16}$.

On the other hand, all is well with less large values, like $10^{15}$:

In [32]:
A3a = [1.0  1e15 ; 1.0  1.0]
b3a = [1.0 + 1e15 ; 2.0]

println("A3a is"); printmatrix(A3a)
println("b3a is $b3a")

A3a is
[ 1.0 1.0e15 
  1.0 1.0 ]
b3a is [1.000000000000001e15, 2.0]


In [33]:
(U3a, c3a) = rowreduce(A3a, b3a)
x3a = backwardsubstitution(U3a, c3a)

println("U3a is"); printmatrix(U3a)
println("c3a is $c3a")
println("x3a is $x3a")

U3a is
[ 1.0 1.0e15 
  0.0 -9.99999999999999e14 ]
c3a is [1.000000000000001e15, -9.99999999999999e14]
x3a is [1.0, 1.0]


```{prf:example} Avoiding small denominators
:label: example-avoiding-small-denominators

The first equation in {prf:ref}`example-almost-division-by-zero` can be divided by $10^{16}$ to get an equivalent system with the same problem:

$$
\left[\begin{array}{rr} 10^{-16} & 1 \\ 1 & 1 \end{array}\right]
\left[\begin{array}{r} x_1 \\ x_2 \end{array}\right] = \left[\begin{array}{r} 1+10^{-16} \\ 2 \end{array}\right]
$$

Now the problem is more obvious: this system differs from the system in {prf:ref}`example-obvious-division-by-zero`
just by a tiny change of $10^{-16}$ in that pivot elements $a_{11}$, and the problem is *division by a value very close to zero*.
```

In [34]:
A4 = [1e-16  1.0 ; 1.0 1.0]
b4 = [1.0 + 1e-16 ; 2.0]

println("A4 is"); printmatrix(A4)
println("b4 is $b4")

A4 is
[ 1.0e-16 1.0 
  1.0 1.0 ]
b4 is [1.0, 2.0]


In [35]:
(U4, c4) = rowreduce(A4, b4)
x4 = backwardsubstitution(U4, c4)

println("U4 is"); printmatrix(U4)
println("c4 is $c4")
println("x4 is $x4")

U4 is
[ 1.0e-16 1.0 
  0.0 -1.0e16 ]
c4 is [1.0, -9.999999999999998e15]
x4 is [2.220446049250313, 0.9999999999999998]


One might think that there is no such small denominator in {prf:ref}`example-almost-division-by-zero`,
but what counts for being "small" is magnitude relative to other values — 1 is very small compared to $10^{16}$.

To understand these problems more (and how to avoid them) we will explore {doc}`machine-numbers-rounding-error-and-error-propagation` in the next section.

## When naive Guassian elimination is safe: diagonal dominance

There are several important cases when we can guarantee that these problem do not occur.
One obvious case is when the matrix $A$ is diagonal and non-singular (so with all non-zero elements);
then it is already row-reduced and with all denominators in backward substitution being non-zero.

A useful measure of being "close to diagonal" is *diagonal dominance*:

```{prf:definition} Strict Diagonal Dominance
:label: definition-strictly-diagonally-dominant

A matrix $A$ is **row-wise strictly diagonally dominant**,
sometimes abbreviated as just **strictly diagonally dominant** or **SDD**,
if 

$$\sum_{1 \leq k \leq n, k \neq i}|a_{i,k}| < |a_{i,i}|$$

Loosely, each main diagonal "dominates" in size over all other elements in its row.
```

````{prf:definition} Column-wise Strict Diagonal Dominance
:label: definition-columnwise-strictly-diagonally-dominant

If instead

$$\sum_{1 \leq k \leq n, k \neq i}|a_{k,i}| < |a_{i,i}|$$
(so that each main diagonal element "dominates its column")
the matrix is called **column-wise strictly diagonally dominant**.

Note that this is the same as saying that the transpose $A^T$ is SDD.
````

**Aside:** If only the corresponding non-strict inequality holds, the matrix is called *diagonally dominant*.

```{prf:theorem}
:label: theorem-row-reduction-preserves-sdd

For any strictly diagonally dominant matrix $A$, each of the intermediate matrices $A^{(k)}$ given by the naive Gaussan elimination algorithm is also strictly diagonally dominant, and so the final upper triangular matrix $U$ is.
In particular, all the diagonal elements $a_{i,i}^{(k)}$ and $u_{i,i}$ are non-zero, so no division by zero occurs in any of these algorithms, including the backward substitution solving for $x$ in $Ux = c$.

The corresponding fact also true if the matrix is column-wise strictly diagonally dominant: that property is also preserved at each stage in naive Guassian elimination.
```

Thus in each case the diagonal elements — the elements divided by in both row reduction and backward substitution — are in some sense safely away from zero.
We will have more to say about this in the sections on {doc}`linear-equations-2-pivoting`
and {doc}`linear-equations-3-lu-factorization`

For a column-wise SDD matrix, more is true: at stage $k$, the diagonal dominance says that
the pivot elemet on the diagonal, $a_{k,k}^{(k-1)}$, is larger (in magnitude) than any of the elements $a_{i,k}^{(k-1)}$ below it, so the multipliers $l_{i,k}$ have

$$|l_{i,k}| = |a_{i,k}^{(k-1)}/a_{k,k}^{(k-1)}| < 1.$$

As we will see when we look at the effects of rounding error in {doc}`machine-numbers-rounding-error-and-error-propagation` and
{doc}`linear-equations-5-error-bounds-condition-numbers`,
keeping intermediate value small is generally good for accuracy, so this is a nice feature.

```{prf:remark} Positive definite matrices
:label: remark-positive-definite-matrices-also-work

Another class of matrices for which naive Gaussian elimination works well is **positive definite matrices** which arise in any important situations; that property is in some sense more natural than diagonal dominance.
However that topic will be left for later.
```